### Постановка задачи
Посчитаем модель линейной регрессии по первым 100 зданиям и найдем ее точность, используя в качестве параметров только дни недели и праздники, применяя fit_intercept=False и логарифмируя целевой показатель.

Для вычисления отсутствующих или некорректных данных построим модели по всем зданиям одного типа в одном городе и во всех городах.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Загрузка данных, отсечение 100 зданий, объединение и оптимизация

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[(energy["building_id"]<100)]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)
del buildings
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 56.89 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864557 entries, 0 to 864556
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           864557 non-null  datetime64[ns]
 1   site_id             864557 non-null  int8          
 2   building_id         864557 non-null  int8          
 3   meter_reading       864557 non-null  float16       
 4   primary_use         864557 non-null  category      
 5   air_temperature     864263 non-null  float16       
 6   cloud_coverage      487693 non-null  float16       
 7   dew_temperature     864263 non-null  float16       
 8   precip_depth_1_hr   864459 non-null  float16       
 9   sea_level_pressure  856210 non-null  float16       
 10  wind_direction      839970 non-null  float16       
 11  wind_speed          864557 non-null  float16       
dtypes: category(1), datetime64[ns](1)

### Обогащение данных: час, дни недели, праздники, логарифм

In [4]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Разделение данных

In [5]:
energy_train, energy_test = train_test_split(energy[(energy["meter_reading"]>0)], test_size=0.2)
print (energy_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427872 entries, 335314 to 502354
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           427872 non-null  datetime64[ns]
 1   site_id             427872 non-null  int8          
 2   building_id         427872 non-null  int8          
 3   meter_reading       427872 non-null  float16       
 4   primary_use         427872 non-null  category      
 5   air_temperature     427866 non-null  float16       
 6   cloud_coverage      249072 non-null  float16       
 7   dew_temperature     427866 non-null  float16       
 8   precip_depth_1_hr   427869 non-null  float16       
 9   sea_level_pressure  425640 non-null  float16       
 10  wind_direction      414143 non-null  float16       
 11  wind_speed          427872 non-null  float16       
 12  hour                427872 non-null  int8          
 13  weekday             4278

### Линейная регрессия: по часам

In [6]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_lr = [[]]*len(buildings)
for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_lr[building].append([0]*(len(lr_columns)-3))
        energy_train_bh = pd.DataFrame(energy_train_b[energy_train_b["hour"]==hour])
        y = energy_train_bh["meter_reading_log"]
        if len(y) > 0:
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            energy_lr[building][hour] = model.coef_
            energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)
print (energy_lr[0])

[array([-0.07945387,  5.43875882,  5.47690217,  5.4807581 ,  5.4230219 ,
        5.46670483,  5.45442708,  5.471875  ,  0.        ]), array([-0.0914005 ,  5.4448008 ,  5.48234954,  5.50255927,  5.50178102,
        5.47054377,  5.44908854,  5.45517113,  0.        ]), array([-0.04627894,  5.43646713,  5.45401278,  5.4998653 ,  5.45445424,
        5.44453837,  5.46875   ,  5.4958044 ,  0.        ]), array([-0.1424579 ,  5.4939034 ,  5.50455729,  5.46918403,  5.46646735,
        5.42450225,  5.4375    ,  5.45330256,  0.        ]), array([-0.05388532,  5.39962181,  5.44566761,  5.49669471,  5.44629664,
        5.46754808,  5.429375  ,  5.45447198,  0.        ]), array([-0.18384792,  5.47225046,  5.46234375,  5.46424279,  5.50310304,
        5.46813517,  5.44372106,  5.48980978,  0.        ]), array([-0.11465608,  5.50571753,  5.45265625,  5.49667245,  5.44928422,
        5.4515625 ,  5.44984375,  5.42307692,  0.        ]), array([-0.1214097 ,  5.50700295,  5.43875   ,  5.49819712,  5.474687

### Линейная регрессия: по типам зданий

In [7]:
sites = range(0, energy["site_id"].max() + 1)
primary_uses = energy["primary_use"].unique()
lr_columns_use = ["meter_reading_log", "hour", "building_id",
                  "is_holiday", "primary_use", "site_id"]
for wday in range(0,7):
    lr_columns_use.append("is_wday" + str(wday))
energy_lr_use = {}
energy_lr_use_site = {}
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns_use)
for primary_use in primary_uses:
    energy_train_u = energy_train_lr[energy_train_lr["primary_use"]==primary_use]
    if len(energy_train_u) > 0:
        energy_lr_use_site[primary_use] = [[]]*len(sites)
        for site in sites:
            energy_lr_use_site[primary_use][site] = [[]]*len(hours)
            energy_train_us = energy_train_u[energy_train_u["site_id"]==site]
            if len(energy_train_us) > 0:
                for hour in hours:
                    energy_train_uth = energy_train_us[energy_train_us["hour"]==hour]
                    y = energy_train_uth["meter_reading_log"]
                    if len(y) > 0:
                        x = energy_train_uth.drop(labels=["meter_reading_log",
                            "hour", "building_id", "site_id",
                            "primary_use"], axis=1)
                        model = LinearRegression(fit_intercept=False).fit(x, y)
                        energy_lr_use_site[primary_use][site][hour] = model.coef_
                        energy_lr_use_site[primary_use][site][hour] = np.append(energy_lr_use_site[primary_use][site][hour], model.intercept_)
        energy_lr_use[primary_use] = [[]]*len(hours)
        for hour in hours:
            energy_train_th = energy_train_u[energy_train_u["hour"]==hour]
            y = energy_train_th["meter_reading_log"]
            if len(y) > 0:
                x = energy_train_th.drop(labels=["meter_reading_log",
                    "hour", "building_id", "site_id", "primary_use"], axis=1)
                model = LinearRegression(fit_intercept=False).fit(x, y)
                energy_lr_use[primary_use][hour] = model.coef_
                energy_lr_use[primary_use][hour] = np.append(energy_lr_use[primary_use][hour], model.intercept_)
print (energy_lr_use_site)

{'Education': [[array([-0.03887348,  5.61569515,  5.7003147 ,  5.64693707,  5.68380911,
        5.66768302,  5.65309819,  5.61601297,  0.        ]), array([-0.01635904,  5.58822103,  5.65356209,  5.67860462,  5.66931009,
        5.64055029,  5.64747646,  5.61029389,  0.        ]), array([-0.08762793,  5.63344662,  5.64197727,  5.64103386,  5.63661102,
        5.62649051,  5.61585168,  5.57696083,  0.        ]), array([-0.15262148,  5.5906132 ,  5.61058112,  5.65509115,  5.64697504,
        5.64127574,  5.6070033 ,  5.55773826,  0.        ]), array([-0.05852851,  5.56650754,  5.64120723,  5.65522012,  5.63114333,
        5.64634168,  5.61885712,  5.58104666,  0.        ]), array([-0.06378522,  5.60713851,  5.65780439,  5.638647  ,  5.6668637 ,
        5.65008864,  5.63958685,  5.64286725,  0.        ]), array([-0.0310855 ,  5.70151916,  5.74495989,  5.68227283,  5.73677499,
        5.70831502,  5.68440233,  5.71294488,  0.        ]), array([-0.10775594,  5.80253731,  5.89726208,  5.8200

### Расчет качества
Используем индивидуальные модели здания, иначе общую модель по всем зданиям данного типа в городе, иначе общую модель по всем зданиям такого типа (по всем городам)

In [8]:
def calculate_model (x):
    lr = -1
    model = energy_lr[x.building_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use_site[x.primary_use][x.site_id][x.hour]
    if len(model) == 0:
        model = energy_lr_use[x.primary_use][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии, 100 зданий:",
       energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии, 100 зданий: 0.3428103148489524 0.3
